In [18]:
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
import pickle
import numpy as np
from collections import namedtuple
from gensim.models import doc2vec
from keras.models import Sequential
from keras.layers import Dense, Dropout
from bp_mll_keras import bp_mll_loss

pos_tagger = Twitter()

Using TensorFlow backend.


In [2]:
with open('../input/x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)
with open('../input/x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)
with open('../input/y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)
with open('../input/y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
stop_dr = "../input/stopword.txt"
f = open(stop_dr, 'r')
stopword_korean = set([value.strip() for value in f.readlines()])

In [4]:
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [5]:
x_train  = [[i.split("/")[0] for i in tokenize(j) if i.split("/")[1] in ['Noun','Verb','Adjective'] and i.split("/")[0] not in stopword_korean ]for j in x_train ]
#x_train = [','.join(i) for i in x_train]

x_test  = [[i.split("/")[0] for i in tokenize(j) if i.split("/")[1] in ['Noun','Verb','Adjective'] and i.split("/")[0] not in stopword_korean ]for j in x_test ]
#x_test = [','.join(i) for i in x_test]

In [6]:
train_docs = [(token, np.where(label==1.0)[0][0]) for token, label in zip(x_train,y_train)]

In [7]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]

In [28]:
doc_vectorizer = doc2vec.Doc2Vec(vector_size=200, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

for epoch in range(15):
    doc_vectorizer.train(tagged_train_docs,total_examples=doc_vectorizer.corpus_count,epochs=doc_vectorizer.epochs)
    doc_vectorizer.alpha -= 0.002 
    doc_vectorizer.min_alpha = doc_vectorizer.alpha

In [29]:
train_vector = np.array([doc_vectorizer.infer_vector(i) for i in x_train])
test_vector = np.array([doc_vectorizer.infer_vector(i) for i in x_test])

In [31]:
n = train_vector.shape[0]
dim_no = train_vector.shape[1]
class_no = y_train.shape[1]

model = Sequential()
model.add(Dense(512, input_dim=dim_no, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.5))
model.add(Dense(class_no, activation='sigmoid', kernel_initializer='glorot_uniform'))
model.compile(loss=bp_mll_loss, optimizer='adam', metrics=['accuracy'])

model.fit(train_vector, y_train, epochs=2000,validation_split=0.1)
predict = model.predict(test_vector)

Train on 1343 samples, validate on 150 samples
Epoch 1/2000
1343/1343 [==============================] - 1s 389us/step - loss: 26.9069 - acc: 0.2993 - val_loss: 22.4846 - val_acc: 0.3200
Epoch 2/2000
1343/1343 [==============================] - 0s 112us/step - loss: 23.7849 - acc: 0.3291 - val_loss: 21.6686 - val_acc: 0.3333
Epoch 3/2000
1343/1343 [==============================] - 0s 112us/step - loss: 23.0925 - acc: 0.3254 - val_loss: 21.2157 - val_acc: 0.3333
Epoch 4/2000
1343/1343 [==============================] - 0s 112us/step - loss: 22.4698 - acc: 0.3269 - val_loss: 20.7014 - val_acc: 0.3333
Epoch 5/2000
1343/1343 [==============================] - 0s 112us/step - loss: 21.7452 - acc: 0.3522 - val_loss: 19.9357 - val_acc: 0.3867
Epoch 6/2000
1343/1343 [==============================] - 0s 112us/step - loss: 21.1245 - acc: 0.3455 - val_loss: 19.3971 - val_acc: 0.4133
Epoch 7/2000
1343/1343 [==============================] - 0s 112us/step - loss: 20.8562 - acc: 0.3418 - val_loss:

Epoch 59/2000
1343/1343 [==============================] - 0s 112us/step - loss: 18.0878 - acc: 0.3931 - val_loss: 17.8654 - val_acc: 0.4333
Epoch 60/2000
1343/1343 [==============================] - 0s 114us/step - loss: 18.1241 - acc: 0.3939 - val_loss: 17.6172 - val_acc: 0.4333
Epoch 61/2000
1343/1343 [==============================] - 0s 113us/step - loss: 18.1491 - acc: 0.3835 - val_loss: 17.7724 - val_acc: 0.4267
Epoch 62/2000
1343/1343 [==============================] - 0s 113us/step - loss: 18.1051 - acc: 0.3902 - val_loss: 17.8809 - val_acc: 0.4133
Epoch 63/2000
1343/1343 [==============================] - 0s 112us/step - loss: 18.0595 - acc: 0.4080 - val_loss: 17.9798 - val_acc: 0.4267
Epoch 64/2000
1343/1343 [==============================] - 0s 111us/step - loss: 18.1172 - acc: 0.4036 - val_loss: 17.7618 - val_acc: 0.4200
Epoch 65/2000
1343/1343 [==============================] - 0s 112us/step - loss: 17.9784 - acc: 0.3909 - val_loss: 17.7831 - val_acc: 0.4200
Epoch 66/2000

Epoch 117/2000
1343/1343 [==============================] - 0s 112us/step - loss: 16.5462 - acc: 0.5495 - val_loss: 17.0853 - val_acc: 0.5667
Epoch 118/2000
1343/1343 [==============================] - 0s 111us/step - loss: 16.8799 - acc: 0.5391 - val_loss: 16.7453 - val_acc: 0.5600
Epoch 119/2000
1343/1343 [==============================] - 0s 113us/step - loss: 16.7770 - acc: 0.5361 - val_loss: 16.8588 - val_acc: 0.5467
Epoch 120/2000
1343/1343 [==============================] - 0s 111us/step - loss: 16.5957 - acc: 0.5421 - val_loss: 16.6598 - val_acc: 0.5400
Epoch 121/2000
1343/1343 [==============================] - 0s 112us/step - loss: 16.5090 - acc: 0.5562 - val_loss: 16.5480 - val_acc: 0.5733
Epoch 122/2000
1343/1343 [==============================] - 0s 112us/step - loss: 16.6587 - acc: 0.5577 - val_loss: 16.5218 - val_acc: 0.5800
Epoch 123/2000
1343/1343 [==============================] - 0s 111us/step - loss: 16.7826 - acc: 0.5398 - val_loss: 16.4901 - val_acc: 0.5800
Epoch 

Epoch 175/2000
1343/1343 [==============================] - 0s 112us/step - loss: 15.9329 - acc: 0.5957 - val_loss: 16.3822 - val_acc: 0.6000
Epoch 176/2000
1343/1343 [==============================] - 0s 111us/step - loss: 15.9370 - acc: 0.5860 - val_loss: 16.4550 - val_acc: 0.5800
Epoch 177/2000
1343/1343 [==============================] - 0s 114us/step - loss: 15.9851 - acc: 0.5875 - val_loss: 16.4668 - val_acc: 0.6000
Epoch 178/2000
1343/1343 [==============================] - 0s 113us/step - loss: 15.8572 - acc: 0.6091 - val_loss: 16.6449 - val_acc: 0.6067
Epoch 179/2000
1343/1343 [==============================] - 0s 113us/step - loss: 15.9680 - acc: 0.5853 - val_loss: 16.4558 - val_acc: 0.5933
Epoch 180/2000
1343/1343 [==============================] - 0s 112us/step - loss: 15.9149 - acc: 0.5949 - val_loss: 16.4642 - val_acc: 0.5867
Epoch 181/2000
1343/1343 [==============================] - 0s 112us/step - loss: 15.9663 - acc: 0.5905 - val_loss: 16.5388 - val_acc: 0.5933
Epoch 

Epoch 233/2000
1343/1343 [==============================] - 0s 113us/step - loss: 15.5721 - acc: 0.6188 - val_loss: 16.3377 - val_acc: 0.6200
Epoch 234/2000
1343/1343 [==============================] - 0s 113us/step - loss: 15.5092 - acc: 0.6418 - val_loss: 16.4521 - val_acc: 0.6200
Epoch 235/2000
1343/1343 [==============================] - 0s 112us/step - loss: 15.4570 - acc: 0.6240 - val_loss: 16.5978 - val_acc: 0.6200
Epoch 236/2000
1343/1343 [==============================] - 0s 116us/step - loss: 15.4753 - acc: 0.6381 - val_loss: 16.3104 - val_acc: 0.6200
Epoch 237/2000
1343/1343 [==============================] - 0s 114us/step - loss: 15.3695 - acc: 0.6277 - val_loss: 16.3624 - val_acc: 0.6200
Epoch 238/2000
1343/1343 [==============================] - 0s 114us/step - loss: 15.4729 - acc: 0.6314 - val_loss: 16.3991 - val_acc: 0.6133
Epoch 239/2000
1343/1343 [==============================] - 0s 113us/step - loss: 15.5865 - acc: 0.6299 - val_loss: 16.3460 - val_acc: 0.6267
Epoch 

...

1343/1343 [==============================] - 0s 113us/step - loss: 13.4393 - acc: 0.8280 - val_loss: 19.0230 - val_acc: 0.5933
Epoch 1444/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5812 - acc: 0.8258 - val_loss: 18.4415 - val_acc: 0.6067
Epoch 1445/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.7174 - acc: 0.8265 - val_loss: 18.5415 - val_acc: 0.6067
Epoch 1446/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.5037 - acc: 0.8191 - val_loss: 18.5404 - val_acc: 0.5933
Epoch 1447/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.6411 - acc: 0.8153 - val_loss: 18.1423 - val_acc: 0.5933
Epoch 1448/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5737 - acc: 0.8153 - val_loss: 18.3989 - val_acc: 0.5867
Epoch 1449/2000
1343/1343 [==============================] - 0s 111us/step - loss: 13.5575 - acc: 0.8168 - val_loss: 18.1015 - val_acc: 0.6200
Epoch 1450/2000

1343/1343 [==============================] - 0s 112us/step - loss: 13.7304 - acc: 0.8042 - val_loss: 18.3516 - val_acc: 0.5867
Epoch 1558/2000
1343/1343 [==============================] - 0s 111us/step - loss: 13.5672 - acc: 0.8325 - val_loss: 18.5672 - val_acc: 0.5867
Epoch 1559/2000
1343/1343 [==============================] - 0s 111us/step - loss: 13.6270 - acc: 0.8340 - val_loss: 18.2749 - val_acc: 0.5867
Epoch 1560/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5796 - acc: 0.8332 - val_loss: 18.2373 - val_acc: 0.6067
Epoch 1561/2000
1343/1343 [==============================] - 0s 111us/step - loss: 13.6805 - acc: 0.8168 - val_loss: 18.3595 - val_acc: 0.5867
Epoch 1562/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.6148 - acc: 0.8354 - val_loss: 18.6296 - val_acc: 0.5867
Epoch 1563/2000
1343/1343 [==============================] - 0s 116us/step - loss: 13.8143 - acc: 0.8325 - val_loss: 18.5164 - val_acc: 0.5933
Epoch 1564/2000

1343/1343 [==============================] - 0s 113us/step - loss: 13.6104 - acc: 0.8317 - val_loss: 18.6781 - val_acc: 0.5733
Epoch 1672/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5038 - acc: 0.8280 - val_loss: 18.4621 - val_acc: 0.6267
Epoch 1673/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.4974 - acc: 0.8503 - val_loss: 18.4281 - val_acc: 0.6067
Epoch 1674/2000
1343/1343 [==============================] - 0s 114us/step - loss: 13.4389 - acc: 0.8369 - val_loss: 18.6537 - val_acc: 0.5933
Epoch 1675/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.4986 - acc: 0.8369 - val_loss: 18.6689 - val_acc: 0.5667
Epoch 1676/2000
1343/1343 [==============================] - 0s 114us/step - loss: 13.3930 - acc: 0.8414 - val_loss: 18.9884 - val_acc: 0.5733
Epoch 1677/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.5461 - acc: 0.8340 - val_loss: 18.6369 - val_acc: 0.5933
Epoch 1678/2000

1343/1343 [==============================] - 0s 110us/step - loss: 13.6951 - acc: 0.8377 - val_loss: 18.7135 - val_acc: 0.6000
Epoch 1786/2000
1343/1343 [==============================] - 0s 114us/step - loss: 13.5977 - acc: 0.8325 - val_loss: 18.5539 - val_acc: 0.5867
Epoch 1787/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.5067 - acc: 0.8511 - val_loss: 18.7656 - val_acc: 0.6000
Epoch 1788/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5495 - acc: 0.8258 - val_loss: 18.6740 - val_acc: 0.6000
Epoch 1789/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.5334 - acc: 0.8340 - val_loss: 18.3592 - val_acc: 0.6133
Epoch 1790/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.4856 - acc: 0.8503 - val_loss: 18.6587 - val_acc: 0.6067
Epoch 1791/2000
1343/1343 [==============================] - 0s 112us/step - loss: 13.6875 - acc: 0.8124 - val_loss: 18.6596 - val_acc: 0.6067
Epoch 1792/2000

1343/1343 [==============================] - 0s 113us/step - loss: 13.6110 - acc: 0.8496 - val_loss: 18.7106 - val_acc: 0.6000
Epoch 1900/2000
1343/1343 [==============================] - 0s 111us/step - loss: 13.5541 - acc: 0.8481 - val_loss: 18.6867 - val_acc: 0.5867
Epoch 1901/2000
1343/1343 [==============================] - 0s 114us/step - loss: 13.4347 - acc: 0.8421 - val_loss: 18.5957 - val_acc: 0.6200
Epoch 1902/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.4829 - acc: 0.8451 - val_loss: 19.0446 - val_acc: 0.6000
Epoch 1903/2000
1343/1343 [==============================] - 0s 114us/step - loss: 13.3825 - acc: 0.8421 - val_loss: 18.6375 - val_acc: 0.5867
Epoch 1904/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.5854 - acc: 0.8436 - val_loss: 18.7272 - val_acc: 0.5933
Epoch 1905/2000
1343/1343 [==============================] - 0s 113us/step - loss: 13.5076 - acc: 0.8518 - val_loss: 18.3852 - val_acc: 0.6000
Epoch 1906/2000

In [32]:
for standard in [x * 0.1 for x in range(1, 10)]:
    print("Probability: ", standard)
    print(classification_report(y_test==1, predict>standard))

Probability:  0.1
             precision    recall  f1-score   support

          0       0.54      0.61      0.57       161
          1       0.41      0.31      0.35        55
          2       0.00      0.00      0.00         1
          3       0.33      0.52      0.40       240
          4       0.00      0.00      0.00        17
          5       0.00      0.00      0.00         1
          6       0.55      0.82      0.66       566
          7       0.30      0.52      0.38        75
          8       0.31      0.39      0.35       170
          9       0.34      0.56      0.42       199
         10       0.49      0.75      0.59       455

avg / total       0.45      0.65      0.53      1940

Probability:  0.2
             precision    recall  f1-score   support

          0       0.56      0.60      0.58       161
          1       0.44      0.29      0.35        55
          2       0.00      0.00      0.00         1
          3       0.34      0.50      0.40       240
      

/home/ubuntu/anaconda3/envs/ml_python/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
